In [1]:
from pymilvus import connections,db

In [2]:
conn = connections.connect(host="127.0.0.1", port=19530)

In [3]:
database = db.create_database("courseDesign")

RPC error: [create_database], <MilvusException: (code=65535, message=database already exist: courseDesign)>, <Time:{'RPC start': '2024-12-25 20:03:11.175894', 'RPC error': '2024-12-25 20:03:11.194375'}>


MilvusException: <MilvusException: (code=65535, message=database already exist: courseDesign)>

In [4]:
print(db.list_database())

['test', 'default', 'courseDesign']


由于这是我第一次使用milvus，接下来验证一下库操作是否成功,创建test库，在test上进行collection操作

In [5]:
from pymilvus import connections,db

conn = connections.connect(host="127.0.0.1", port=19530)
database = db.create_database("test")

RPC error: [create_database], <MilvusException: (code=65535, message=database already exist: test)>, <Time:{'RPC start': '2024-12-25 20:03:27.707084', 'RPC error': '2024-12-25 20:03:27.711581'}>


MilvusException: <MilvusException: (code=65535, message=database already exist: test)>

In [6]:
print(db.list_database())

['default', 'courseDesign', 'test']


In [7]:
from pymilvus import MilvusClient
client = MilvusClient(uri='http://localhost:19530',db_name='test')

In [8]:
# 判断是否存在
if not client.has_collection(collection_name="demo_collection"):
    client.create_collection(collection_name="demo_collection",dimension=768)
# 因为在ollama_test中我查看了faiss的维度为768，所以在这里也设定为768

#查看db下的collection列表
print(client.list_collections())

['demo_collection', 'collection_name1', 'quick_setup']


In [9]:
res = client.describe_collection(collection_name="demo_collection")
print(res)

{'collection_name': 'demo_collection', 'auto_id': False, 'num_shards': 1, 'description': '', 'fields': [{'field_id': 100, 'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'params': {}, 'is_primary': True}, {'field_id': 101, 'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'aliases': [], 'collection_id': 454829125289199001, 'consistency_level': 2, 'properties': {}, 'num_partitions': 1, 'enable_dynamic_field': True}


In [10]:
# 删除collection
client.drop_collection(collection_name="demo_collection")

In [11]:
print(client.list_collections())

['collection_name1', 'quick_setup']


可以看到，collection类似mysql中的表
下面通过指定数段的方式创建collection

In [13]:
from pymilvus import MilvusClient
from pymilvus import FieldSchema, CollectionSchema, DataType
client = MilvusClient(uri='http://localhost:19530',db_name='test')

id_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, description="primary id")
age_field = FieldSchema(name="age", dtype=DataType.INT64, description="age")
embedding_field = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=128, description="vector") 
position_field = FieldSchema(name="position", dtype=DataType.VARCHAR, max_length=256, is_partition_key=True)
schema = CollectionSchema(fields=[id_field, age_field, position_field,embedding_field], auto_id=False, enable_dynamic_field=True, description="desc of a collection")

#创建collection
collection_name1 = "test_collection1"
client.create_collection(collection_name="collection_name1", schema=schema, dimension=768,)


数据的增加更新和删除

In [14]:
from pymilvus import MilvusClient
from pymilvus import model

client = MilvusClient(uri='http://localhost:19530',db_name='test')

if not client.has_collection("quick_setup"):
    client.create_collection(
        collection_name="quick_setup",
        dimension=5,
        metric_type="IP"
    )
data=[
    {"id": 0, "vector": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, -0.26286205330961354, 0.9029438446296592], "color": "pink_8682"},
    {"id": 1, "vector": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, 0.2614474506242501, 0.838729485096104], "color": "red_7025"},
    {"id": 2, "vector": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, 0.7894058910881185, 0.20785793220625592], "color": "orange_6781"},
    ]

res = client.insert(
    collection_name="quick_setup",
    data=data
)
print(res)

{'insert_count': 3, 'ids': [0, 1, 2]}


In [15]:
res = client.delete(collection_name="quick_setup", filter="id in [0, 1, 2]")

milvus已跑通，完成测试